In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import numpy as np
from PIL import Image
import glob

images = []
for file in glob.glob('./Segmented/Right/*.bmp'):
    image = Image.open(file)
    numpy_array = np.array(image)
    images.append(numpy_array)
for file in glob.glob('./Segmented/Left/*.bmp'):
    image = Image.open(file)
    numpy_array = np.array(image)
    images.append(numpy_array)

palm_arrays = np.array(images)

In [ ]:
x_train = palm_arrays[:2000]
x_test = palm_arrays[2000:]
print(x_train.shape)
print(x_test.shape)

In [ ]:
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
x_train = x_train.reshape(-1, 150, 150, 1)
x_test = x_test.reshape(-1, 150, 150, 1)
print(x_train.shape)
print(x_test.shape)

In [ ]:
import tensorflow as tf
from tensorflow import keras

input_img = keras.Input(shape=(150,150,1)) # shape = (width, height, channels)
encoded = keras.layers.Conv2D(64, (3,3), activation='relu', padding='valid')(input_img)
encoded = keras.layers.MaxPooling2D((2,2))(encoded)
encoded = keras.layers.Conv2D(128, (3,3), activation='relu', padding='same')(encoded)
encoded = keras.layers.MaxPooling2D((3,3))(encoded)
encoded = keras.layers.Conv2D(512, (3,3), activation='relu', padding='same')(encoded)
encoded = keras.layers.Flatten()(encoded)
encoded = keras.layers.Dense(24*24, activation='relu')(encoded)
encoded = keras.layers.Dense(128, activation='relu')(encoded)
encoded = keras.layers.Dense(64, activation='relu')(encoded)
encoded = keras.layers.Dense(32, activation='relu')(encoded)

decoded = keras.layers.Dense(32, activation='relu')(encoded)
decoded = keras.layers.Dense(64, activation='relu')(encoded)
decoded = keras.layers.Dense(128, activation='relu')(decoded)
decoded = keras.layers.Dense(25*25, activation='relu')(decoded)
decoded = keras.layers.Reshape((25,25,1))(decoded)
decoded = keras.layers.Conv2D(512, (3,3), activation='relu', padding='same')(decoded)
decoded = keras.layers.UpSampling2D((3,3))(decoded)
decoded = keras.layers.Conv2D(128, (3,3), activation='relu', padding='same')(decoded)
decoded = keras.layers.UpSampling2D((2,2))(decoded)


autoencoder = keras.Model(input_img, decoded)

In [ ]:
# Encoder Model
encoder = keras.Model(input_img, encoded)

In [ ]:
autoencoder.compile(optimizer='adam', loss=tf.keras.losses.mae)

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.fit(x_train, x_train, epochs=2, shuffle=True)

In [ ]:
encoded_palms = encoder.predict(x_test)

In [ ]:
encoded_palms = encoder.predict(x_test)